# Functions

In [22]:
import chess

squares_index = {
  'a': 0,
  'b': 1,
  'c': 2,
  'd': 3,
  'e': 4,
  'f': 5,
  'g': 6,
  'h': 7
}


# example: h3 -> 17
def square_to_index(square):
  letter = chess.square_name(square)
  return 8 - int(letter[1]), squares_index[letter[0]]


def split_dims(board):
  # this is the 3d matrix
  board3d = numpy.zeros((14, 8, 8), dtype=numpy.int8)

  # here we add the pieces's view on the matrix
  for piece in chess.PIECE_TYPES:
    for square in board.pieces(piece, chess.WHITE):
      idx = numpy.unravel_index(square, (8, 8))
      board3d[piece - 1][7 - idx[0]][idx[1]] = 1
    for square in board.pieces(piece, chess.BLACK):
      idx = numpy.unravel_index(square, (8, 8))
      board3d[piece + 5][7 - idx[0]][idx[1]] = 1

  # add attacks and valid moves too
  # so the network knows what is being attacked
  aux = board.turn
  board.turn = chess.WHITE
  for move in board.legal_moves:
      i, j = square_to_index(move.to_square)
      board3d[12][i][j] = 1
  board.turn = chess.BLACK
  for move in board.legal_moves:
      i, j = square_to_index(move.to_square)
      board3d[13][i][j] = 1
  board.turn = aux

  return board3d

In [24]:
best_move = None


def minimaxv0(board, depth, alpha, beta, maximizingPlayer, model, original_depth):
  global best_move
  if depth == 0 or board.is_game_over():
    return minimax_eval(board, model)

  if maximizingPlayer:
    value = -numpy.inf
    for move in board.legal_moves:
      board.push(move)

      if depth == original_depth:
        temp_value = value
        value = max(value, minimaxv0(board, depth - 1, alpha,
                    beta, False, model, original_depth))
        if value != temp_value:
          best_move = move
      else:
        value = max(value, minimaxv0(board, depth - 1, alpha,
                    beta, False, model, original_depth))

      board.pop()
      if value >= beta:
        break
      alpha = max(alpha, value)
    return value

  else:
    value = numpy.inf
    for move in board.legal_moves:
      board.push(move)
      value = min(value, minimaxv0(board, depth - 1, alpha,
                  beta, True, model, original_depth))
      board.pop()
      if value <= alpha:
        break
      beta = min(beta, value)
    return value


def minimax_eval(board, model):
  board3d = split_dims(board)
  board3d = numpy.expand_dims(board3d, 0)
  return model.predict(board3d)[0][0]

board = chess.Board()
move = minimaxv0(board, 1, -numpy.inf, numpy.inf, True, model, 3)
print(move)
print(best_move)


0.50292844
None


In [29]:
import numpy
import tensorflow.keras.callbacks as callbacks


def get_dataset():
	# Download dataset.npz from the link in Readme.md
	container = numpy.load('dataset.npz')
	b, v = container['b'], container['v']
	v = numpy.asarray(v / abs(v).max() / 2 + 0.5,
	                  dtype=numpy.float32)  # normalization (0 - 1)
	return b, v


x, y = get_dataset()


In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(
    x, y, test_size=0.3, random_state=42)


# TensorFlow!

In [33]:
"""
Initialize Tensorflow model
"""
import numpy
import tensorflow

# cen_model = tensorflow.keras.models.load_model('model_weight/global_model.h5')
fed0_model = tensorflow.keras.models.load_model('model_weight/global_model.h5')
fed1_model = tensorflow.keras.models.load_model('model_weight/global_model.h5')
fed2_model = tensorflow.keras.models.load_model('model_weight/global_model.h5')
# cen_model.tensorflow.keras.models.load_model("model_weight/centralized_weights")
fed0_model.load_weights("model_weight/federated0_weights")
fed1_model.load_weights("model_weight/federated1_weights")
fed2_model.load_weights("model_weight/federated2_weights")


# Checking Accuracy

In [34]:
results0 = fed0_model.evaluate(X_test, Y_test, batch_size = 128)
print("test loss, test acc:", results0)

3516/3516 [==============================] - 54s 15ms/step - loss: 2.7958e-04
test loss, test acc: 0.0002795836189761758


In [35]:
results1 = fed1_model.evaluate(X_test, Y_test, batch_size=128)
print("test loss, test acc:", results1)


3516/3516 [==============================] - 59s 17ms/step - loss: 2.7958e-04
test loss, test acc: 0.00027957966085523367


In [36]:
results2 = fed2_model.evaluate(X_test, Y_test, batch_size=128)
print("test loss, test acc:", results2)

3516/3516 [==============================] - 55s 16ms/step - loss: 0.0010
test loss, test acc: 0.001041704323142767


In [ ]:
# this function will create our x (board)
import random


def random_board(max_depth=200):
  board = chess.Board()
  depth = random.randrange(0, max_depth)

  for _ in range(depth):
    all_moves = list(board.legal_moves)
    random_move = random.choice(all_moves)
    board.push(random_move)
    if board.is_game_over():
      break

  return board

In [ ]:
board = random_board()
move = get_ai_move(board, 2, True, model)
board.push(move)
board


In [ ]:
board.pop()

move = minimaxv0(board,2, -numpy.inf, numpy.inf, True, model, 2)
print(best_move)
board.push(best_move)
board


In [ ]:
board.pop()
move = get_ai_move0(board, 2, True, model)
print(move)
board.push(move)
board


In [ ]:
board.pop()
move = minimax2(board, model)
print(move)
board.push(move)
board


In [ ]:
board

In [ ]:
def minimax2(board, model):
    beta = -numpy.inf
    stop = False
    for move in board.legal_moves:
        mini_eval = numpy.inf
        board.push(move)
        
        for move1 in board.legal_moves:
            if stop:
                pass
            else:
                board.push(move1)
                eval = minimax_eval(board, model)
                board.pop()
                if(eval < beta):
                    stop = True
                    # beta = eval
                mini_eval = min(mini_eval, eval) # Best move for black
        
        if stop == False:
            beta = mini_eval
            print(beta)
            best_move = move
        else:
            stop = False
        board.pop()
    return best_move


def minimax3(board, model):
    alpha = numpy.inf
    stop = False
    for move in board.legal_moves:
        max_eval = -numpy.inf
        board.push(move)
        
        for move1 in board.legal_moves:
            if stop:
                pass
            else:
                board.push(move1)
                eval = minimax_eval(board, model)
                board.pop()
                if(eval > alpha):
                    stop = True
                max_eval = max(max_eval, eval)  # Best move for White
        
        if stop == False:
            alpha = max_eval
            best_move = move
            print(alpha)
        else:
            stop = False
        board.pop()
    return best_move


def minimax_eval(board, model):
    board3d = split_dims(board)
    board3d = numpy.expand_dims(board3d, 0)
    return model.predict(board3d)[0][0]


In [ ]:
"""
AI vs AI
"""
board = chess.Board()

while True:
    move = get_ai_move(board, 1, True, model)
    # move = minimax2(board, model)
    board.push(move)
    print(move)
    print(f'\n{board}')
    if board.is_game_over():
        print('game_over')
        break
    # move = get_ai_move(board, 1, False, model)
    move = minimax3(board, model)
    board.push(move)
    print(move)
    print(f'\n{board}')
    if board.is_game_over():
        print('game_over')
        break


In [ ]:
board